In [1]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

df = pd.read_json('datasets/HotPotQA/hotpot_dev_distractor_v1.json')

## Experiment


### Graph creation and refinement section

To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [2]:
context_list = df['context']

In [3]:
df.head(21)

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Ed Wood (film), [Ed Wood is a 1994 American ...",comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[Meet Corliss Archer, [Meet Corliss Archer, a...",bridge,hard
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...","[[Andre Norton Award, [The Andre Norton Award ...",bridge,hard
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]","[[Esma Sultan (daughter of Abdülaziz), [Esma S...",comparison,hard
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...","[[Just Another Romantic Wrestling Comedy, [Jus...",bridge,hard
5,5abd94525542992ac4f382d2,YG Entertainment,2014 S/S is the debut album of a South Korean ...,"[[2014 S/S, 0], [Winner (band), 0]]",[[List of awards and nominations received by S...,bridge,hard
6,5a85b2d95542997b5ce40028,Eenasul Fateh,Who was known by his stage name Aladin and hel...,"[[Eenasul Fateh, 0], [Management consulting, 0]]","[[James P. Comer, [James P. Comer (born James ...",bridge,hard
7,5a87ab905542996e4f3088c1,"3,677 seated",The arena where the Lewiston Maineiacs played ...,"[[Lewiston Maineiacs, 1], [Androscoggin Bank C...","[[Billings Bulls, [The Billings Bulls were a j...",bridge,hard
8,5a7bbb64554299042af8f7cc,Terry Richardson,"Who is older, Annie Morton or Terry Richardson?","[[Annie Morton, 0], [Annie Morton, 2], [Terry ...","[[Annie Morton, [Annie Morton (born October 8,...",bridge,hard
9,5a8db19d5542994ba4e3dd00,yes,Are Local H and For Against both from the Unit...,"[[Local H, 0], [For Against, 0]]","[[Mendocino County GMO Ban, [Mendocino County,...",comparison,hard


In [ ]:
df.iloc[7]['context']

In [6]:
context_list[11:21]

11    [[Ungod, [Ungod is the debut album released in...
12    [[Arnold Schwarzenegger filmography, [Arnold S...
13    [[Real estate benchmarking, [Real estate bench...
14    [[Joshua Parlby, [Joshua Parlby (born 1855 in ...
15    [[Fishing Lake, [Fishing Lake is a lake in the...
16    [[2010–11 Vermont Catamounts men's basketball ...
17    [[Bernhard Bötel, [Bernhard Bötel (1883–1953) ...
18    [[United States Assistant Secretary of State, ...
19    [[Asgard (yacht), [Asgard is a 51 ft gaff rigg...
20    [[2009 Singapore Grand Prix, [The 2009 Singapo...
Name: context, dtype: object

In [7]:
propositions = []

for i, value in enumerate(tqdm(context_list[11:21])):
        for j, context in enumerate(value):
                index = 1
                if index < len(context):
                    words = context[1]
                    combined = " ".join(words)
                    get_propositions_nosplit(combined, propositions)

100%|██████████| 10/10 [15:37<00:00, 93.72s/it]


In [8]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

['Ungod is the debut album released in 1994 on Columbia Records.', 'Ungod was released by the American industrial rock band Stabbing Westward.', 'The album was recorded in six weeks of 1993 in Chiswick, England.', 'Ungod was released on February 15, 1994.', 'Adequate album sales and touring with the likes of Depeche Mode warranted a second album.', "The second album is titled 'Wither Blister Burn & Peel'.", "The guitar line in the chorus of the song 'Ungod' was later used in Filter's song 'Hey Man, Nice Shot.'", 'Stuart Zechman was playing guitar for Filter at the time.', "Stuart Zechman took the riff from the song 'Ungod' and showed it to Stabbing Westward.", 'Stabbing Westward ended up using the riff as well.', "The song 'Nothing' appeared in the movie 'Bad Boys'.", "The movie 'Bad Boys' starred Will Smith, Martin Lawrence, and Téa Leoni.", "The song 'Nothing' was not featured on the official soundtrack album of 'Bad Boys'.", "The song 'Nothing' accompanied the credits of the film 'J

In [9]:
list_of_edges = generateEdges(propositions_from_file)

100%|██████████| 1192/1192 [52:34<00:00,  2.65s/it] 


In [10]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

Success


In [ ]:
qg = QueryGraph()

if refine(qg, "person"):
    print("yes")
else:
    print("no")

### Q and A section

In [ ]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [12]:
qg = QueryGraph()
my_dict = {}

# Define a function to apply to each row
def print_qa(row):
    
    question = row['question']
    questionid = row['_id']
    req = qg.get_requirements(question)
    result = qg.answer_question(question, req.content)

    if result is not None:
    
        model_answer = result['result']

        if "don't know the answer" in model_answer:
            model_answer = ""
    
        if len(row['answer']) > 0:
            real_answer = row['answer']
        else:
            real_answer = "\"\""

        if len(result['intermediate_steps']) > 0:
            cypher_query = result['intermediate_steps'][0]['query']
            context = result['intermediate_steps'][1]['context']
       
            if(len(context) < 1):
            #     result = refine_query(cypher_query[6:], question)
            #     model_answer = result['result'] 
                model_answer = "\"\""
                my_dict.update({questionid: ""})
            else:
                my_dict.update({questionid: model_answer})
                
        with open("preds.txt", "a") as preds:
            preds.write("question: " + question + "\n")
            preds.write("real_answer: " + real_answer + "\n")
            preds.write("model_answer: " + model_answer + "\n")  
            preds.write("cypher_query: " + cypher_query + "\n")
            preds.write("======================" + "\n")  
        preds.close()

    else: 
        my_dict.update({questionid: ""})
        

In [13]:
my_dict = {}
# Apply the function to each row
df[11:21].progress_apply(print_qa, axis=1)
    
with open("dev_fullwiki_pred.json", "a") as json_file:
    json.dump(my_dict, json_file, indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(evolution)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(evolution)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(nicolas cage|téa leoni)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(nicolas cage|téa leoni)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description
Full Context:
[]


 20%|██        | 2/10 [00:05<00:23,  3.00s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(guns n roses)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(guns n roses)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(arnold schwarzenegger)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(arnold schwarzenegger)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(former new york police detective)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(former new york police detective)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description
Full Context:
[]


 30%|███       | 3/10 [00:20<00:54,  7.83s/it]

...


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(henry roth)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(henry roth)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(robert erskine childers)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(robert erskine childers)\\b.*'
)
AND (
    toLower(r1.metadata) =~ '.*\\b(england)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(england)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description
Full Context:
[]


100%|██████████| 10/10 [01:10<00:00,  6.80s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(force india\\s+driver\\s+born\\s+in\\s+1990)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(force india\\s+driver\\s+born\\s+in\\s+1990)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(mexican\\s+formula\\s+one\\s+driver\\s+podium)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(mexican\\s+formula\\s+one\\s+driver\\s+podium)\\b.*'
)
RETURN DISTINCT r1.metadata, r1.description, r2.metadata, r2.description
Full Context:
[]


100%|██████████| 10/10 [01:21<00:00,  8.12s/it]


> Finished chain.


In [12]:
x = df.iloc[20]
question = x['question']
print("question:", question)
answer = x['answer']
print("answer:", answer)

qg = QueryGraph()
req = qg.get_requirements(question)
res = qg.answer_question(question, req.content)
res

question: Which other Mexican Formula One race car driver has held the podium besides the Force India driver born in 1990?
answer: Pedro Rodríguez


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) =~ '.*\\b(force india\\s*driver\\s*born\\s*in\\s*1990)\\b.*' OR
    toLower(r1.description) =~ '.*\\b(force india\\s*driver\\s*born\\s*in\\s*1990)\\b.*'
)
AND (
    toLower(r2.metadata) =~ '.*\\b(mexican\\s*formula\\s*one\\s*driver\\s*podium)\\b.*' OR 
    toLower(r2.description) =~ '.*\\b(mexican\\s*formula\\s*one\\s*driver\\s*podium)\\b.*'
)
RETURN DISTINCT r2.metadata, r2.description
Full Context:
[]

> Finished chain.


{'query': 'Which other Mexican Formula One race car driver has held the podium besides the Force India driver born in 1990?',
 'requirements': '1. Key components of the question:\n   - Identification of a Mexican Formula One race car driver.\n   - Reference to a specific driver born in 1990 who drove for Force India.\n   - Inquiry about other drivers who have held a podium position.\n\n2. Requirements needed to answer the question:\n   - Name of the Force India driver born in 1990.\n   - List of Mexican Formula One drivers who have held a podium position.\n   - Information on podium finishes of Mexican drivers in Formula One.\n\n3. Additional context or data needed:\n   - Historical data on Formula One podium finishes by Mexican drivers.',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': "MATCH (w)-[r1]-(x)-[r2]-(y)\nWHERE (\n    toLower(r1.metadata) =~ '.*\\\\b(force india\\\\s*driver\\\\s*born\\\\s*in\\\\s*1990)\\\\b.*' OR\n    toLower(r1.description) =~ '.*\\

## Graph refinement 

In [ ]:
qg = QueryGraph()

QUERY = """
MATCH (N) RETURN N
"""
res = qg._graph.query(QUERY)
res

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [ ]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

Create embeddings

In [ ]:
# # Now we initialize from existing graph
# existing_graph = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
#     url=neo4j_url,
#     username=neo4j_user,
#     password=neo4j_password,
#     index_name="person_index",
#     node_label="Person",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

Refactor node labels

In [ ]:
# # cq = """
# # MATCH (en:Entity)
# # WHERE en.label = 'Person'
# # WITH collect(en) AS persons
# # CALL apoc.refactor.rename.label("Entity", "Person", persons)
# # YIELD batches, total, timeTaken, committedOperations
# # RETURN batches, total, timeTaken, committedOperations;
# # """
# # res = qg._graph.query(cq)

Create projection

In [ ]:
cq = """
MATCH (p:Person)
RETURN gds.graph.project(
  'personproj',
  p,
  null,
  {
    sourceNodeProperties: p { .embedding },
    targetNodeProperties: {}
  }
)
"""

res = qg._graph.query(cq)

Use KNN to measure node similarities

In [ ]:
cq = """
CALL gds.knn.stream('projection2', {
topK: 1,
nodeProperties: ['embedding'],
// The following parameters are set to produce a deterministic result
randomSeed: 1337,
concurrency: 1,
sampleRate: 1.0,
deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE similarity > .90
RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

res = qg._graph.query(cq)
res

In [ ]:
for i in res:
    for key, value in i.items():
        print(f"{key}: {value}")

In [ ]:
def are_dictionaries_equivalent(d1, d2):
    # Check if both have the same keys
    if d1.keys() != d2.keys():
        return False

    # Check if Person1 and Person2 are swapped but equivalent
    if (d1['Person1'] == d2['Person2'] and d1['Person2'] == d2['Person1'] and d1['similarity'] == d2['similarity']):
        return True
    
    return False


In [ ]:
newlist = res

In [ ]:
for idx, val in enumerate(res):   
    for idx2, val2 in enumerate(newlist):
        if val != val2:
            if are_dictionaries_equivalent(val, val2):
                newlist.pop(idx2)


In [ ]:
unique = []

for value in newlist:
    if value not in unique:
        unique.append(value)

unique

In [ ]:
node_name = "Shirley Temple"

query = f"""
MATCH (n:Person {{name: '{node_name}'}})
RETURN n
"""
print(query)

res = qg._graph.query(query)
res


Merge similar nodes (Entity resolution)

In [ ]:
for idx, val in enumerate(unique):
    print(val)
    p1 = val['Person1']
    p2 = val['Person2']    
    cq = f"""
    MATCH (a1:Person {{name: '{p1}'}}), (a2:Person {{name: '{p2}'}})
    WITH head(collect([a1,a2])) as nodes
    CALL apoc.refactor.mergeNodes(nodes,{{
    properties:"discard",
    mergeRels:true
    }})
    YIELD node
    RETURN node;
    """
    res = qg._graph.query(cq)
    res